In [1]:
import cv2
import numpy as np
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:

class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5t prohibited",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]


In [3]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v02.pt")

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    annotated_frame = results[0].plot()

    cv2.imshow("Traffic Sign Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 222.0ms
Speed: 7.2ms preprocess, 222.0ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 173.9ms
Speed: 3.3ms preprocess, 173.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 148.8ms
Speed: 2.3ms preprocess, 148.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 141.9ms
Speed: 2.1ms preprocess, 141.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 136.3ms
Speed: 3.2ms preprocess, 136.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 142.9ms
Speed: 1.8ms preprocess, 142.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 140.3ms
Speed: 2.3ms preprocess, 140.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 128.6ms
Speed: 2.1ms prepr

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8_fine_tuned_v02.pt")
model.info()

Model summary: 129 layers, 11,152,241 parameters, 0 gradients, 28.7 GFLOPs


(129, 11152241, 0, 28.73728)

In [5]:
print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [6]:

image_folder = "PreparedData/images/test"
labels_folder = "PreparedData/labels/test"

output_file = "v2_test_pred.txt"


with open(output_file, "w") as f:

    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(image_folder, img_name)
            results = model.predict(source=img_path, save=False, conf=0.25, verbose=False)

            
            class_ids = results[0].boxes.cls.tolist()

            f.write(f"{img_name} " + " ".join(str(int(cls_id)) for cls_id in class_ids) + "\n")



In [7]:
with open(output_file, "r") as f:
    lines = f.readlines()
pred = []
for line in lines:
    parts = line.strip().split(' ')
    img_name = parts[0]
    if len(parts) > 1:
        class_ids = [int(cls_id) for cls_id in parts[1:]]
    else:
        class_ids = [99]
    if class_ids:
        pred.append((img_name, class_ids))
actual = []
for file in os.listdir(labels_folder):
    with open(os.path.join(labels_folder, file), "r") as f:
        content = f.readlines()
        content = [line.strip() for line in content if line.strip()]
        if content:
            class_id = []
            for line in content:
                class_id.append(int(line.split()[0]))
            actual.append((file, class_id))   
sum = 0
count = 0

for i in range(len(actual)):
    count += (max(len(actual[i][1]), len(pred[i][1])))

for i in range(len(actual)):
    for j in pred[i][1]:
        if j in actual[i][1]:
            sum += 1
accuracy = sum / count
accuracy

0.9880546075085325